In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# pacotes para executar operações gráfricas,tabulares,matemáticas

import numpy as np 
import pandas as pd 
import scipy.io as sio
import matplotlib.pyplot as plt
import seaborn as sns
from skimage.io import imshow
from pylab import rcParams

!pip install missingpy

#pacotes para 'imputar' dados
import missingpy
from missingpy import MissForest
from sklearn.impute import SimpleImputer

In [ ]:
Impute_data=pd.read_excel("/kaggle/input/impute/Impute.xlsx")#ler conjunto de dados via pandas

Impute_data #mostrar

In [ ]:
#executar funções de 'imputar'

RFORImp=MissForest()
SKLSimple=SimpleImputer(missing_values=np.nan,strategy='median')

Simple_Imp = Impute_data.fillna(method='ffill')
SKL_Imp = pd.DataFrame(SKLSimple.fit_transform(Impute_data))
RFOR_Tab = pd.DataFrame(RFORImp.fit_transform(Impute_data))#Random Forest Imputação e conversão por pandas



In [ ]:
#crear gráficos

plt.plot(Simple_Imp.index,Simple_Imp ['Sedimentation'], label='Simple_Imp Sedimentation', linewidth=3, color='midnightblue')
plt.plot(Simple_Imp.index,Simple_Imp ['Sedimentation_in_Dead_Storage'], label='Simple_Imp Sediment in Dead_Volume', linewidth=3, color='orange')
plt.plot(SKL_Imp.index,SKL_Imp [0], label='Median_Imp Sedimentation', linewidth=3, color='darkgreen')
plt.plot(SKL_Imp.index,SKL_Imp [1], label ='Median_Imp Sediment in Dead_Volume', linewidth=3, color='red')
plt.plot(RFOR_Tab.index,RFOR_Tab [0], label='RFOR_Tab Sedimentation', linewidth=3, color='blueviolet')
plt.plot(RFOR_Tab.index,RFOR_Tab [1], label ='RFOR_Tab Sediment in Dead_Volume', linewidth=3, color='saddlebrown')

plt.legend(loc='right',fontsize=12)


rcParams['figure.figsize'] = 16, 6


plt.show()

In [ ]:

# pacotes escencias por certas funciones como randomizar ou gerar certas representações gráficas 
import os
import cv2
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix


# pacotes gerais de ANN
import tensorflow as tf 
from tensorflow import keras


#from tensorflow.keras import datasets, layers, models

# pacotes por certas operções ou câmaras de ANN
from tensorflow.keras import optimizers, losses, utils 
from tensorflow.keras.layers import Dense, Activation, Conv2D, MaxPool2D, Flatten, Dropout
from tensorflow.keras.models import Model, Sequential 
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.regularizers import l2



In [ ]:
DATADIR = '/kaggle/input/eurosat/2750/' #lugar superior do conjunto de imagens
CATEGORIES = ["River","Forest","AnnualCrop","HerbaceousVegetation","Highway","Industrial","Pasture","PermanentCrop","Residential","SeaLake"]#conjuntos das categórias

training_data = [] #lista vazia
show = []

#rotação sobre Datadir e Categorias para a)marcar cada categoria com um número único b)unificar as imanges num arquivo só  
        

def create_image_set():
    for category in CATEGORIES:
        path = os.path.join(DATADIR, category)
        class_num = CATEGORIES.index(category)  
        for img in os.listdir(path):
            try: # try-except-pass: medida de segurança contra arquivos quebrados
                img_array=cv2.imread(os.path.join(path,img))
                training_data.append([img_array,class_num])
                show.append(img_array)
            except Exception as e:
                pass
#             plt.imshow(img_array)
#             plt.show()
#             break
#         break

create_image_set()

In [ ]:
print(plt.imshow(show[10930]))

In [ ]:
print(training_data)

In [ ]:
random.shuffle(training_data) #randomizar ordem das imagens

In [ ]:
#crear duas listas. X: Imagens existentes. y: Marcas existentes

X = []
y = []

for features, label in training_data:
    X.append(features)
    y.append(label)
    
#crear marcação de One-Hot-Encoding
    
ye=pd.DataFrame(y)
    
New=pd.DataFrame(columns=CATEGORIES)
ye=pd.concat([ye, New],axis=1)

ye.loc[ye['River'].isna()==True, ('River','Forest','AnnualCrop','HerbaceousVegetation','Highway','Industrial','Pasture','PermanentCrop','Residential','SeaLake')] = 0
ye.loc[ye[0] ==0, 'River'] = 1
ye.loc[ye[0] ==1, 'Forest'] = 1
ye.loc[ye[0] ==2, 'AnnualCrop'] = 1
ye.loc[ye[0] ==3, 'HerbaceousVegetation'] = 1
ye.loc[ye[0] ==4, 'Highway'] = 1
ye.loc[ye[0] ==5, 'Industrial'] = 1
ye.loc[ye[0] ==6, 'Pasture'] = 1
ye.loc[ye[0] ==7, 'PermanentCrop'] = 1
ye.loc[ye[0] ==8, 'Residential'] = 1
ye.loc[ye[0] ==9, 'SeaLake'] = 1

ye=ye.drop(ye.columns[0], axis=1)                                    

#mudar pelo tipo de dado float32 que permite numeros do conjunto real
y=ye.astype('float32')      
print(y)



#conversão dos conjunto num numpy.array, necessario para executar a ANN    
X = np.array(X)
y = np.array(y)

In [ ]:
print(y.shape,X.shape)
print(np.isnan(y).any()) #controlar, se existem NaN (valores non-númericos) dentro do conjunto

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=196) #crear um conjuntos para treinar e testar

In [ ]:
#reservar os últimos 2500 imagens por validar durante o treinamento

X_val = X_train[-2500:]
y_val = y_train[-2500:]
X_train = X_train[:-2500]
y_train = y_train[:-2500]


In [ ]:
print (X_train)

In [ ]:
input_shape=[64, 64, 3] #forma dum imagen utilizada: 64pixel*64pixel*3cores

model = Sequential() #modo de escrever a ANN

model.add(Conv2D(28, (3, 3), padding='same',input_shape=input_shape)) 
model.add(Activation('relu')) 
model.add(Conv2D(28, (3, 3), padding='same')) 
model.add(Activation('relu')) 
model.add(MaxPool2D(2,2)) 

model.add(Conv2D(56, (3, 3), padding='same')) 
model.add(Activation('relu')) 
model.add(Conv2D(56, (3, 3), padding='same')) 
model.add(Activation('relu')) 
model.add(MaxPool2D(2,2))

model.add(Conv2D(112, (3, 3), padding='same')) 
model.add(Activation('relu')) 
model.add(Conv2D(112, (3, 3), padding='same')) 
model.add(Activation('relu')) 
model.add(MaxPool2D(2,2)) 

model.add(Flatten()) 
model.add(Dense(784)) 
model.add(Activation('relu')) 
model.add(Dropout(0.6)) 
model.add(Dense(10)) 

model.add(Activation('sigmoid')) 
adam = optimizers.Adam(lr=0.001) 
model.compile(optimizer=adam, loss=losses.binary_crossentropy, metrics=['accuracy'])


          
print(model.summary())

In [ ]:
#Entreinamento

history = model.fit(X_train, y_train, epochs=15,
                    validation_data=(X_val, y_val))


In [ ]:
# Curvas do Loss
plt.figure(figsize=[8,6])
plt.plot(history.history['loss'],'r',linewidth=3.0)
plt.plot(history.history['val_loss'],'b',linewidth=3.0)
plt.legend(['Training loss', 'Validation Loss'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Loss',fontsize=16)
plt.title('Loss Curves',fontsize=16)
 
# Curvas da precisão
plt.figure(figsize=[8,6])
plt.plot(history.history['accuracy'],'r',linewidth=3.0)
plt.plot(history.history['val_accuracy'],'b',linewidth=3.0)
plt.legend(['Training Accuracy', 'Validation Accuracy'],fontsize=18)
plt.xlabel('Epochs ',fontsize=16)
plt.ylabel('Accuracy',fontsize=16)
plt.title('Accuracy Curves',fontsize=16)

In [ ]:
# Predição por X_test

preds2 = model.predict(X_test, verbose=1)

In [ ]:
#a matrix de confusão, mostrando o rácio de sucesso da ANN


print(y_test.shape)
y_test= np.array([np.argmax(y) for y in y_test])
y_predict=np.array([ np.argmax(p) for p in preds2])
print(y_test.shape)
print(y_predict.shape)
confusion=confusion_matrix(y_test, y_predict)
sns.heatmap(confusion, annot=True, annot_kws={"size": 14}, fmt='g', linewidths=.5, 
            yticklabels='r''f''a''v''h''i''p''c''d''s',xticklabels='r''f''a''v''h''i''p''c''d''s')
plt.ylabel('True label')
plt.xlabel('Predicted label')